In [2]:
from utils.vasp.vasp import DatabaseGenerator
import argparse
import warnings
from multiprocessing import cpu_count
import os

/root/personal_python_utilities/utils/vasp/parser/outcar.py:227: SyntaxWarning: invalid escape sequence '\A'
  """
/root/personal_python_utilities/utils/vasp/parser/outcar.py:257: SyntaxWarning: invalid escape sequence '\A'
  """
/root/personal_python_utilities/utils/vasp/parser/outcar.py:284: SyntaxWarning: invalid escape sequence '\A'
  """
/root/personal_python_utilities/utils/vasp/parser/outcar.py:312: SyntaxWarning: invalid escape sequence '\A'
  """
/root/personal_python_utilities/utils/vasp/parser/outcar.py:1077: SyntaxWarning: invalid escape sequence '\A'
  """
/root/personal_python_utilities/utils/vasp/parser/outcar.py:1118: SyntaxWarning: invalid escape sequence '\A'
  """


In [7]:
directory = "/root/Fe_Ptable_Project/128_Fe_Bulk_data/Fe-Ptable-Data/Bulk"
datagen = DatabaseGenerator(directory,
                            max_workers=cpu_count())
# Call the build_database function with the updated parameters
df = datagen.build_database(extract_directories=False,
                            read_multiple_runs_in_dir=False,
                            read_error_dirs=False,
                            max_dir_count=1000,
                            tarball_extensions=(".tar.gz", ".tar.bz2"),
                            cleanup=False,
                            keep_filenames_after_cleanup=[],
                            keep_filename_patterns_after_cleanup=[],
                            filenames_to_qualify=["OUTCAR"],
                            df_filename=None,
                            df_compression=True,
                            )


The total number of vasp directories that we are building the database out of is 90
# Processes: 90, Processors available: 16, CPUs used: 16


ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''
ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''
ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''
ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''
ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''
ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''
ufunc 'isnan' not supported for the input types, and the inputs could not be

In [9]:
df.to_pickle(os.path.join(os.getcwd(), f"vasp_database.pkl.gz"), compression="gzip")

In [11]:
import os
import glob
import time

from pymatgen.core import Structure
from pymatgen.io.vasp import Vasprun, Kpoints, Incar

import numpy as np
import pandas as pd

import utils.generic as gen_tools
from utils.parallel import parallelise
from utils.vasp.parser.outcar import Outcar
from utils.vasp.vasp_database import parse_vasp_directory

def update_failed_jobs_in_database(parent_dir, df_path=None, read_error_dirs=False, read_multiple_runs_in_dir=False, max_dir_count=None, df_compression=True, max_workers=16):
    compression_option = 'gzip' if df_compression else None
    compression_extension = '.gz' if df_compression else ''
    
    if df_path is None:
        df_path = os.path.join(parent_dir, f"vasp_database.pkl{compression_extension}")
    
    if os.path.isdir(df_path):
        potential_files = [
            os.path.join(df_path, "vasp_database.pkl.gz"),
            os.path.join(df_path, "vasp_database.pkl")
        ]
        output_path = os.path.join(df_path, f"vasp_database.pkl{compression_extension}")
    else:
        potential_files = [df_path]
        output_path = df_path
    
    df = None
    for file in potential_files:
        try:
            if file.endswith(".gz"):
                df = pd.read_pickle(file, compression='gzip')
            else:
                df = pd.read_pickle(file, compression=None)
            print(f"Successfully read database from {file}")
            break
        except (FileNotFoundError, pd.errors.UnrecognizedCompressionError):
            print(f"Failed to read database from {file}")

    if df is None:
        raise ValueError("Invalid path or filename - please check! Attempted paths: " + ", ".join(potential_files))
    
    failed_dirs = df[df['convergence'] == False]['filepath'].tolist()
    print(f"Reparsing {len(failed_dirs)} directories where convergence is False")

    if max_dir_count:
        pkl_filenames = []
        for i, chunks in enumerate(gen_tools.chunk_list(failed_dirs, max_dir_count)):
            step_time = time.time()
            failed_df = pd.concat(parallelise(parse_vasp_directory, 
                                              [(chunk,) for chunk in chunks],
                                              max_workers=max_workers,
                                              extract_error_dirs=read_error_dirs, 
                                              parse_all_in_dir=read_multiple_runs_in_dir))
            db_filename = f"update_{i}.pkl{compression_extension}"
            pkl_filenames.append(os.path.join(parent_dir, db_filename))
            failed_df.to_pickle(os.path.join(parent_dir, db_filename), compression=compression_option)
            step_taken_time = np.round(time.time() - step_time, 3)
            print(f"Step {i}: {step_taken_time} seconds taken for {len(chunks)} parse steps")
        
        failed_df = pd.concat([pd.read_pickle(partial_df, compression=compression_option) for partial_df in pkl_filenames])
    else:
        failed_df = pd.concat(parallelise(parse_vasp_directory, 
                                          [(chunk,) for chunk in failed_dirs],
                                          max_workers=max_workers,
                                          extract_error_dirs=read_error_dirs, 
                                          parse_all_in_dir=read_multiple_runs_in_dir))

    # # Ensure no duplicate labels in index before update
    # if failed_df.index.duplicated().any():
    #     failed_df = failed_df[~failed_df.index.duplicated()]

    # if df.index.duplicated().any():
    #     df = df[~df.index.duplicated()]

    # # Use a different method to merge the DataFrames
    # df.update(failed_df, overwrite=True, filter_func=lambda x: not pd.isna(x))

    # df.to_pickle(output_path, compression=compression_option)
    # print(f"Updated dataframe saved to {output_path}")
    return failed_df

In [ ]:


# Call the update_failed_jobs_in_database function with the updated parameters
df = datagen.update_failed_jobs_in_database(df_path=directory,
                                            read_error_dirs=False,
                                            read_multiple_runs_in_dir=False,
                                            max_dir_count=max_dir_count,
                                            df_compression=True)